# Retrieve Catalog Data from `catalogue.json`

The file `catalogue.json` contains all the catalog data for an [ORACC](http://oracc.org) project (for general information, see the [Oracc Open Data](http://oracc.org/doc/opendata) page). The `zip` that contains all JSON files of a particular project can be found at `http://oracc.museum.upenn.edu/[PROJECT]/json/[PROJECT].zip`. In the URL replace [PROJECT] with your project name (e.g. `dcclt`). For sub-projects the URL pattern is `http://oracc.museum.upenn.edu/[PROJECT]/[SUBPROJECT]/json/[PROJECT]-[SUBPROJECT].zip`. 

The main key in a `catalogue.json` file is called `members`. The value of this key contains the information of all the fields and all the entries in the project catalog. This information is put in a `pandas` DataFrame.

In [1]:
import pandas as pd
import zipfile
import json
import os
import sys
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. The directories are created with the function `make_dirs()` from the `utils` module. 

In [2]:
directories = ['jsonzip', 'output']
make_dirs(directories)

## 1.1 Input Project Names
We can download and manipulate multiple [ORACC](http://oracc.org) `zip` files at the same time. Note, however that different [ORACC](http://oracc.org) projects use different fields in their catalogs; not all catalogs are mutually compatible.

Provide project abbreviations, separated by a comma. Note that subprojects must be processed separately, they are not included in the main project. A subproject is named `[PROJECT]/[SUBPROJECT]`, for instance `saao/saa01`.

Split the list of projects and create a list of project names, using the `format_project_list()` function from the `utils` module.

In [3]:
projects = input('Project(s): ').lower().strip() # lowercase user input and strip accidental spaces
p = format_project_list(projects)

Project(s):  dcclt


## 1.2 Split the List of Projects and Download the ZIP files.
Use the `oracc_download()` function from the `utils` module to download the requested projects. The code of this function is discussed in more detail in 2.1.0. Download ORACC JSON Files.

In [4]:
p = oracc_download(p)

Saving http://build-oracc.museum.upenn.edu/json/dcclt.zip as jsonzip/dcclt.zip.


## 2 Extract Catalogue Data from `JSON` files
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file, named `[PROJECT].zip` has been downloaded in the directory `jsonzip` (1.2). Each of these `zip` files includes a file called `catalogue.json`. This file is read in and loaded with the command `json.loads()`, which transforms a string into a JSON object - a sequence of names and values.

The JSON object is transformed into a `pandas` Dataframe. By default, when reading in a dictionary, the `DataFrame()` function will take the top-level keys (in this case the text IDs) as columns. The dataframe needs to be transposed (`.T`), so that the P, Q, and X numbers become indexes or row names, and each column represents a field in the catalog.  The individual dataframes (one for each project requested) are concatenated. Since individual [ORACC](http://oracc.org) project catalogs may have different fields, the dataframes may have different column names. By default `pandas` concatenation uses an `outer join` so that all column names of all the catalogs are preserved.

In [5]:
df = pd.DataFrame() # create an empty dataframe
for project in p:
    file = f"jsonzip/{project.replace('/', '-')}.zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(f"{file} does not exist or is not a proper ZIP file")
        continue
    try:
        st = z.read(f"{project}/catalogue.json").decode('utf-8')  #read and decode the catalogue.json file of one project
                                                                # the result is a string object
    except:
        print(f"{project}/catalogue.json is not available or not complete")
        continue
    cat = json.loads(st)
    cat = cat['members']  # select the 'members' node 
    for item in cat.values():
        item["project"] = project # add project name as separate field
    cat_df = pd.DataFrame(cat).T
    df = pd.concat([df, cat_df], sort=True)  # sort=True is necessary in case catalogs have a different set of fields
df

,accession_no,acquisition_history,archive,ark,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,...,subgenre,subgenre_remarks,supergenre,surface_preservation,text_remarks,thickness,trans,uri,width,xproject
P000001,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,NaN,...,Archaic Lu A,NaN,LEX,NaN,NaN,18,NaN,http://cdli.ucla.edu/P000001,61,CDLI
P000002,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.",30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,NaN,...,Archaic Lu A,NaN,LEX,NaN,NaN,13,NaN,http://cdli.ucla.edu/P000002,48,CDLI
P000003,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,NaN,...,Archaic Vocabulary,NaN,LEX,NaN,NaN,19,NaN,http://cdli.ucla.edu/P000003,53,CDLI
P000004,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.",26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,NaN,...,Archaic Lu A,NaN,LEX,NaN,NaN,23,NaN,http://cdli.ucla.edu/P000004,23,CDLI
P000005,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,NaN,...,Archaic Lu A,NaN,LEX,NaN,NaN,20,NaN,http://cdli.ucla.edu/P000005,36,CDLI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X999901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Murgud,L 2,LEX,NaN,To Ura 7-8,NaN,NaN,NaN,NaN,NaN
X999902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Murgud,L 3,LEX,NaN,To Ura 8-10,NaN,NaN,NaN,NaN,NaN
X999903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Murgud,L 4,LEX,NaN,To Ura 11,NaN,NaN,NaN,NaN,NaN
X999905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Murgud,L 1,LEX,NaN,To Ura 7A-B,NaN,NaN,NaN,NaN,NaN


## 3. Clean the Dataframe
The function `fillna('')` will put a blank (instead of `NaN`) in all fields that have no entry.

In [6]:
df = df.fillna('')
df

,accession_no,acquisition_history,archive,ark,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,...,subgenre,subgenre_remarks,supergenre,surface_preservation,text_remarks,thickness,trans,uri,width,xproject
P000001,,,,21198/zz001q0dtm,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",,,...,Archaic Lu A,,LEX,,,18,,http://cdli.ucla.edu/P000001,61,CDLI
P000002,,,,21198/zz001q0dv4,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.",30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,,,...,Archaic Lu A,,LEX,,,13,,http://cdli.ucla.edu/P000002,48,CDLI
P000003,,,,21198/zz001q0dwn,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",,,...,Archaic Vocabulary,,LEX,,,19,,http://cdli.ucla.edu/P000003,53,CDLI
P000004,,,,21198/zz001q0dx5,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.",26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,,,...,Archaic Lu A,,LEX,,,23,,http://cdli.ucla.edu/P000004,23,CDLI
P000005,,,,21198/zz001q0dzp,"Englund, Robert K.",20130125.atf,"Englund, Robert K. &amp; Nissen, Hans J.","29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",,,...,Archaic Lu A,,LEX,,,20,,http://cdli.ucla.edu/P000005,36,CDLI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X999901,,,,,,,,,,,...,Murgud,L 2,LEX,,To Ura 7-8,,,,,
X999902,,,,,,,,,,,...,Murgud,L 3,LEX,,To Ura 8-10,,,,,
X999903,,,,,,,,,,,...,Murgud,L 4,LEX,,To Ura 11,,,,,
X999905,,,,,,,,,,,...,Murgud,L 1,LEX,,To Ura 7A-B,,,,,


## 4 Select Relevant Fields
[ORACC](http://oracc.org) catalogs may have custom fields, the only fields that are obligatory are `id_text` (the P, Q, or X number that identifies the text, for instance "P243546") and `designation` (the human-readable reference, for instance "VS 17, 012"). The example code below works with field names that are available in (almost) every catalog. Adjust the code to your data and your needs.

In [ ]:
df1 = df[['designation', 'period', 'provenience',
        'museum_no', 'project', 'id_text']]
df1

## 5.1 Save as CSV
Save the resulting data set as a `csv` file. `UTF-8` encoding is the encoding with the widest support in text analysis (and also the encoding used by [ORACC](http://oracc.org)). If you intend to use the catalog file in Excel, however, it is better to use `utf-16` encoding.

In [ ]:
filename = 'catalog.csv'
with open(f'output/{filename}', 'w', encoding='utf-8') as w:
    df1.to_csv(w, index=False)

## 5.2 Save with Pickle
One may pickle a file either with the `pickle` library or directly from within `pandas` with the `to_pickle()` function. A pickled file preserves the data structure of the dataframe, which is an advantage over saving as `csv`. The pickle file is a binary file, so we must open the file with the `wb` (write binary) option and we cannot give an encoding. To open the pickled file one may use the `read_pickle()` function from the `pandas` library, as in:

```python
import pandas as pd
df = pd.read_pickle(o)
```

In [ ]:
filename = "output/catalog.p"
df1.to_pickle(filename)